# Robot Simulation
This simulation is used for data generation for prognostic algorithm testing. The equations of motion are generated automatically using Kane's method via PyDy.

A lot of the content here comes from [this](https://github.com/pydy/pydy-tutorial-human-standing) tutorial by Jason Moore but I'll leave out the learning and use different variable names.

In [1]:
from __future__ import print_function, division
from sympy import symbols, simplify
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, Point
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)

# Reference Frames

In [2]:
inertial_frame = ReferenceFrame('I')
upper_arm_frame = ReferenceFrame('U')
fore_arm_frame = ReferenceFrame('F')
# theta1 - base, theta2 - elbow
theta1, theta2 = dynamicsymbols('theta1, theta2')

In [5]:
upper_arm_frame.orient(inertial_frame, 'Axis', (theta1, inertial_frame.z))
fore_arm_frame.orient(upper_arm_frame, 'Axis' (theta2, upper_arm_frame.z))

# Points

In [6]:
# Joints
shoulder = Point('A')
upper_arm_length = symbols('l_U')

elbow = Point('E')
elbow.set_pos(shoulder, upper_arm_length * upper_arm_frame.y)

end_effector = Point('EE')
fore_arm_length = symbols('l_F')
end_effector.set_pos(elbow, fore_arm_length * fore_arm_frame.y)

In [8]:
# Center of Masses
upper_arm_com_length, fore_arm_com_length = symbols('d_U, d_F')

upper_arm_mass_center = Point('U_o')
upper_arm_mass_center.set_pos(shoulder, upper_arm_com_length * upper_arm_frame.y)

fore_arm_mass_center = Point('F_o')
fore_arm_mass_center.set_pos(elbow, fore_arm_com_length * fore_arm_frame.y)

# Kinematical Differential Equations